---
format:
  html:
    code-line-numbers: false
    code-overflow: wrap
    code-block-bg: true
    code-block-border-left: true
    highlight-style: arrow
---

# Benders Decomposition

In this chapter, we will explain the theories behind Benders decomposition and demonstrate its usage on a trial linear programming problem.
Keep in mind that Benders decomposition is not limited to solving linear programming problems.
In fact, it is one of the most powerful techniques to solve some large-scale mixed-integer linear programming problems.

In the following sections, we will go through the critical steps during the decomposition process when applying the algorithm on optimization problems represented in standard forms.
This is important as it helps build up the intuition of when we should consider applying Benders decomposition to a problem at hand.
Often times, recognizing the applicability of Benders decomposition is the most important and challenging step when solving an optimization problem.
Once we know that the problem structure is suitable to solve via Benders decomposition, it is straightforward to follow the decomposition steps and put it into work.

Generally speaking, Benders decomposition is a good solution candidate when the resulting problem is much easier to solve if some of the variables in the original problem are fixed.
We will illustrate this point using an example in the following sections.
In the optimization world, the first candidate that should come to mind when we say a problem is easy to solve is a linear programming formulation, which is indeed the case in Benders decomposition applications.

## The Decomposition Logic

To explain the workings of Benders decomposition, let us look at the standard form of linear programming problems that involve two vector variables, $\mathbf{x}$ and $\mathbf{y}$. Let $p$ and $q$ indicate the dimensions of $\mathbf{x}$ and $\mathbf{y}$, respectively.
Below is the original problem (**P**) we intend to solve.

\begin{align}
\text{min.} &\quad \mathbf{c}^T \mathbf{x} + \mathbf{f}^T \mathbf{y} \\
\text{s.t.} &\quad \mathbf{A} \mathbf{x} + \mathbf{B} \mathbf{y} = \mathbf{b} \\
&\quad \mathbf{x} \geq 0, \mathbf{y} \geq 0
\end{align}

In this formulation, $\mathbf{c}$ and $\mathbf{f}$ in the objective function represent the cost coefficients associated with decision variables $\mathbf{x}$ and $\mathbf{y}$, respectively.
Both of them are column vectors of corresponding dimensions.
In the constraints, matrix $\mathbf{A}$ is of dimension $m \times p$, and matrix $\mathbf{B}$ is of dimension $m \times q$.
$\mathbf{b}$ is a column vector of dimension $m$.

Suppose the variable $\mathbf{y}$ is a complicating variable in the sense that the resulting problem is substantially easier to solve if the value of $\mathbf{y}$ is fixed.
In this case, we could rewrite problem $\mathbf{P}$ as the following form:

\begin{align}
\text{min.} &\quad \mathbf{f}^T \mathbf{y} + g(\mathbf{y}) \\
\text{s.t.} &\quad \mathbf{y} \geq 0
\end{align}

where $g(\mathbf{y})$ is a function of $\mathbf{y}$ and is defined as the subproblem $\mathbf{SP}$ of the form below:

\begin{align}
    \text{min.} &\quad \mathbf{c}^T \mathbf{x} \\
    \text{s.t.} &\quad \mathbf{A} \mathbf{x}  = \mathbf{b} - \mathbf{B} \mathbf{y} \label{bd-cons1} \\
    &\quad \mathbf{x} \geq 0
\end{align}

Note that the $\mathbf{y}$ in constraint \eqref{bd-cons1} takes on some known values when the problem is solved and the only decision variable in the above formulation is $\mathbf{x}$.
The dual problem of $\mathbf{SP}$, $\mathbf{DSP}$, is given below.


\begin{align}
    \text{max.} &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u} \\
    \text{s.t.} &\quad \mathbf{A}^T \mathbf{u} \leq \mathbf{c} \label{bd-cons2} \\
    &\quad \mathbf{u}\  \text{unrestricted}
\end{align}

A key characteristic of the above $\mathbf{DSP}$ is that its solution space does not depend on the value of $\mathbf{y}$, which only affects the objective function.
According to the Minkowski’s representation theorem, any $\bar{\mathbf{u}}$ satisfying the constraints \eqref{bd-cons2} can be expressed as

\begin{align}
\bar{\mathbf{u}} = \sum_{j \in \mathbf{J}} \lambda_j \mathbf{u}_{j}^{point} + \sum_{k \in \mathbf{K}} \mu_k \mathbf{u}_k^{ray}
\end{align}

where $\mathbf{u}_j^{point}$ and $\mathbf{u}_k^{ray}$ represent an extreme point and extreme ray, respectively.
In addition, $\lambda_j \geq 0$ for all $j \in \mathbf{J}$ and $\sum_{j \in \mathbf{J}}\lambda_j = 1$, and $\mu_k \geq 0$ for all $k \in \mathbf{K}$.
It follows that the $\mathbf{DSP}$ is equivalent to 

\begin{align}
\text{max.} &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} (\sum_{j \in \mathbf{J}} \lambda_j \mathbf{u}_{j}^{point} + \sum_{k \in \mathbf{K}} \mu_k \mathbf{u}_k^{ray}) \\
\text{s.t.} &\quad \sum_{j \in \mathbf{J}}\lambda_j = 1 \\
&\quad \lambda_j \geq 0, \ \forall j \in \mathbf{J} \\
&\quad \mu_k \geq 0, \ \forall k \in \mathbf{K}
\end{align}

We can therefore conclude that

- The $\mathbf{DSP}$ becomes unbounded if any $\mathbf{u}_k^{ray}$ exists such that $(\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_k^{ray} > 0$.
Note that an unbounded $\mathbf{DSP}$ implies an infeasible $\mathbf{SP}$ and to prevent this from happening, we have to ensure that $(\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_k^{ray} \leq 0$ for all $k \in \mathbf{K}$.
- If an optimal solution to $\mathbf{DSP}$ exists, it must occur at one of the extreme points. Let $g$ denote the optimal objective value, it follows that $(\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_j^{point} \leq g$ for all $j \in \mathbf{J}$.

Based on this idea, the $\mathbf{DSP}$ can be reformulated as follows:

\begin{align}
\text{min.} &\quad g \\
\text{s.t.} &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_k^{ray} \leq 0, \ \forall j \in \mathbf{J} \label{bd-feas} \\
&\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_j^{point} \leq g, \ \forall k \in \mathbf{K} \label{bd-opt} \\
&\quad j \in \mathbf{J}, k \in \mathbf{K}
\end{align}

Constraints \eqref{bd-feas} are called **Benders feasibility cuts**, while constraints \eqref{bd-opt} are called **Benders optimality cuts**.
Now we are ready to define the Benders Master Problem ($\mathbf{BMP}$) as follows:

\begin{align}
    \text{min.} &\quad \mathbf{f}^T \mathbf{y} + g \\
    \text{s.t.} &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_k^{ray} \leq 0, \ \forall j \in \mathbf{J} \\
    &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_j^{point} \leq g, \ \forall k \in \mathbf{K} \\
    &\quad j \in \mathbf{J}, k \in \mathbf{K}, \mathbf{y} \geq 0
\end{align}

Typically $J$ and $K$ are too large to enumerate upfront and we have to work with subsets of them, denoted by $J_s$ and $K_s$, respectively. Hence we have the following Restricted Benders Master Problem ($\mathbf{RBMP}$):

\begin{align}
    \text{min.} &\quad \mathbf{f}^T \mathbf{y} + g \\
    \text{s.t.} &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_k^{ray} \leq 0, \ \forall j \in \mathbf{J}_s \\
    &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_j^{point} \leq g, \ \forall k \in \mathbf{K}_s \\
    &\quad j \in \mathbf{J}, k \in \mathbf{K}, \mathbf{y} \geq 0
\end{align}

```{mermaid}
%%| label: fig-benders-flowchart
%%| fig-cap: Benders decomposition workflow

flowchart LR
   A[Start] --> B{Is}
    B -->|Yes| C[OK]
    C --> D[Rethink]
    D --> B
    B ---->|No| E[End]
```

## Solving Linear Programming Problems with Benders Decomposition

In this section, we use Benders decomposition to solve several linear programming (LP) problems in order to demonstrate the decomposition logic, especially how the restricted Benders master problem interacts with the subproblem in an iterative approach to reach final optimality.
Most linear programs could be solved efficiently nowadays by either open source or commercial solvers without resorting to any decomposition approaches.
However, by working through the example problems in the following sections, we aim to showcase the implementation details when applying Benders decomposition algorithm on real problems, which helps solidify our understanding of Benders decomposition.
Hopefully, by the end of this chapter, we will build up enough intuition as well as hands-on experience such that we are ready to tackle most involved problems in the following chapters.

In the following sections, we employ several steps to illustrate the problem solving process of Benders decomposition.

- We will first create two linear programming solvers based on Gurobi and SCIP that can solve any linear programs defined in the standard form. They are used in later section to validate the correctness of the solutions produced by Benders decomposition.
- Next, we use a specific linear program and give the corresponding RBMP and DSP to prepare for the implementations.
- Then, we will solve the example linear program step by step by examining the outputs of the RBMP and DSP to decided the next set of actions.
- Futhermore, a holistic Benders decomposition implementation is then developed to solve the example linear program.
- Following the previous step, a more generic Benders decomposition implementation is created.
- Then, we will examine an alternative implementation using Gurobi callback functions.
- We will also provide an implementation based on SCIP.
- In the final section, we will do several benchmarking testing.

### LP solvers based on Gurobi and SCIP

We aim to use Benders decomposition to solve several linear programming problems in the following sections.
To do that, we intentionally decompose the LP problem under consideration into two sets, one set of *complicating* variables and the other set containing the remaining variables.
In order to validate the correctness of the results obtained by Benders decomposition, we implement two additional ways of solving the target linear programming problems directly.
The first option is based on the Gurobi API in python and the other is based on the open source solve SCIP.
The two implementations defined here assume the LP problems under consideration follow the below standard form:

\begin{align}
    \text{min.} &\quad \mathbf{c}^T \mathbf{x} \\
    \text{s.t.} &\quad \mathbf{A} \mathbf{x} = \mathbf{b} \\
    &\quad \mathbf{x} \geq 0
\end{align}

@lst-solver-gurobi defines a solver for LP problems using Gurobi.
It takes three constructor parameters:

- `obj_coeff`: this corresponds to the objective coefficients $\mathbf{c}$.
- `constr_mat`: this refers to the constraint matrix $\mathbf{A}$.
- `rhs`: this is the right-hand side $\mathbf{b}$.

Inside the constructor `__init__()`, a solver environment `_env` is first created and then used to initialize a model object `_model`.
The input parameters are then used to create decision variables `_vars`, constraints `_constrs` and objective function respectively.
The `optimize()` function simply solves the problem and shows the solving status.
Finally, the `clean_up()` function frees up the computing resources.

In [104]:
#| lst-label: lst-solver-gurobi
#| lst-cap: A LP solver based on Gurobi
#| code-line-numbers: true

import gurobipy as gp
from gurobipy import GRB
import numpy as np

class LpSolverGurobi:
    
    def __init__(self, obj_coeff, constr_mat, rhs, verbose=False):
        # initialize environment and model
        self._env = gp.Env('GurobiEnv', empty=True)
        # self._env.setParam('LogToConsole', 1 if verbose else 0)
        self._env.setParam('OutputFlag', 1 if verbose else 0)
        self._env.start()
        self._model = gp.Model(env=self._env, name='GurobiLpSolver')
        
        # prepare data
        self._obj_coeff = obj_coeff
        # print(self._obj_coeff)
        self._constr_mat = constr_mat
        # print(self._constr_mat)
        self._rhs = rhs
        self._num_vars = len(self._obj_coeff)
        self._num_constrs = len(self._rhs)
        
        # create decision variables
        self._vars = self._model.addMVar(self._num_vars, 
                                         vtype=GRB.CONTINUOUS, 
                                         lb=0)
        
        # create constraints
        self._constrs = self._model.addConstr(
            self._constr_mat@self._vars == self._rhs
        )
        
        # create objective
        self._model.setObjective(self._obj_coeff@self._vars, 
                                 GRB.MINIMIZE)
    
    def optimize(self, verbose=False):
        self._model.optimize()
        if self._model.status == GRB.OPTIMAL:
            print(f'Optimal solution found!')
            print(f'Optimal objective = {self._model.objVal:.2f}')
        elif self._model.status == GRB.UNBOUNDED:
            print(f'Model is unbounded!')
        elif self._model.status == GRB.INFEASIBLE:
            print(f'Model is infeasible!')
        else:
            print(f'Unknown error occurred!')
    
    def clean_up(self):
        self._model.dispose()
        self._env.dispose()

@lst-solver-scip presents an LP solver implementation in class `LpSolverSCIP` using SCIP.
The constructor requires the same of parameters as defined in `LpSolverGurobi`.
The model building process is similar with minor changes when required to create decision variables, constraints and the objective function.

In [105]:
#| lst-label: lst-solver-scip
#| lst-cap: A LP solver based on SCIP

import pyscipopt as scip
from pyscipopt import SCIP_PARAMSETTING

class LpSolverSCIP:
    
    def __init__(self, obj_coeff, constr_mat, rhs, verbose=False):
        self._model = scip.Model('LpModel')
        if not verbose:
            self._model.hideOutput()
    
        # create variables
        self._vars = {
            i: self._model.addVar(lb=0, vtype='C')
            for i in range(len(obj_coeff))
        }
        
        # create constraints
        for c in range(len(rhs)):
            expr = [
                constr_mat[c][j] * self._vars.get(j)
                for j in range(len(obj_coeff))
            ]
            self._model.addCons(scip.quicksum(expr) == rhs[c])
            
        # create objective
        obj_expr = [
            obj_coeff[i] * self._vars.get(i)
            for i in range(len(obj_coeff))
        ]
        self._model.setObjective(scip.quicksum(obj_expr), "minimize")
    
    def optimize(self):
        self._model.optimize()
        status = self._model.getStatus()
        if status == "optimal":
            print(f'Optimal solution found!')
            print(f'Optimal objective = {self._model.getObjVal():.2f}')
        elif self._model.status == "unbounded":
            print(f'Model is unbounded!')
        elif self._model.status == "infeasible":
            print(f'Model is infeasible!')
        else:
            print(f'Unknown error occurred!')

@lst-lp-exp1 generates a LP problem with 20 decision variables and 5 constraints.

In [106]:
#| lst-label: lst-lp-exp1
#| lst-cap: A randomly generated LP problem

import numpy as np

np.random.seed(42)
c = np.random.randint(1, 6, size=20)
A = np.random.randint(-10, 12, size=(5, 20))
b = np.random.randint(20, 100, size=5)

@lst-lp-solver-gurobi-test1 solves the generated LP using `LpSolverGurobi` and the solver output shows that an optimal solution was found with objective value of 36.90.

In [107]:
#| lst-label: lst-lp-solver-gurobi-test1
#| lst-cap: Solving the generated LP with Gurobi

lpsolver_gurobi = LpSolverGurobi(obj_coeff=c, constr_mat=A, rhs=b) # <1>
lpsolver_gurobi.optimize()

Optimal solution found!
Optimal objective = 36.90


@lst-lp-solver-scip-test1 solves the same LP problem using SCIP and not surprisingly, the same optimal objective value was found.
This is not exciting, as it only indicates that the two solvers agree on the optimal solution on such a small LP problem as expected.
However, they will become more useful in the following sections when we use them to validate our Benders decomposition results.

In [108]:
#| lst-label: lst-lp-solver-scip-test1
#| lst-cap: Solving the generated LP with SCIP

lpsolver_scip = LpSolverSCIP(obj_coeff=c, constr_mat=A, rhs=b)
lpsolver_scip.optimize()

Optimal solution found!
Optimal objective = 36.90


### A serious LP problem that cannot wait to be decomposed!

With the validation tools available for use, we are ready to solve some serious LP problems using Benders decomposition!
What we have below is a LP problem with five decision variables, three of which are denoted by $\mathbf{x} = (x_1, x_2, x_3)$ and the remaining two variables are denoted by $\mathbf{y} = (y_1, y_2)$.
We assume that $\mathbf{y}$ are the complicating variables.

\begin{align*}
    \text{min.} &\quad 8x_1 + 12x_2 +10x_3 + 15y_1 + 18y_2 \\
    \text{s.t.} &\quad 2x_1 + 3x_2 + 2x_3 + 4y_1 + 5y_2 = 300 \\
    &\quad 4x_1 + 2x_2 + 3x_3 + 2y_1 + 3y_2 = 220 \\
    &\quad x_i \geq 0, \ \forall i = 1, \cdots, 3 \\
    &\quad y_i \geq 0, \ \forall j = 1, 2
\end{align*}

According to the standard LP form presented in the previous section, $\mathbf{c}^T = (8, 12, 10)$, $\mathbf{f}^T = (15, 18)$ and $\mathbf{b}^T = (300, 220)$.
In addition,

\begin{equation*}
\mathbf{A} = 
\begin{bmatrix}
    2 & 3 & 2 \\
    4 & 2 & 3 \\
\end{bmatrix}
\qquad
\mathbf{B} = 
\begin{bmatrix}
    4 & 5 \\
    2 & 3 \\
\end{bmatrix}
\end{equation*}

@lst-lp2 solves the LP problem using the two solvers define above. Both solvers confirm the optimal objective value is 1091.43.
With this information in mind, we will apply Benders decomposition to see if the same optimal solution could be identified or not.

In [109]:
#| lst-label: lst-lp2
#| lst-cap: Solve the LP problem with Gurobi and SCIP

c = np.array([8, 12, 10])
f = np.array([15, 18])
obj_coeff = np.concatenate([c, f])

A = np.array([[2, 3, 2],
     [4, 2, 3]])
B = np.array([[4, 5],
     [2, 3]])
constr_mat = np.concatenate([A, B], axis=1)

rhs = np.array([300, 220])

lpsolver_gurobi = LpSolverGurobi(obj_coeff, constr_mat, rhs, verbose=False)
lpsolver_gurobi.optimize()
# Optimal objective = 1091.43

lpsolver_scip = LpSolverSCIP(obj_coeff, constr_mat, rhs, verbose=False)
lpsolver_scip.optimize()
# Optimal objective = 1091.43

Optimal solution found!
Optimal objective = 1091.43
Optimal solution found!
Optimal objective = 1091.43


### Benders decomposition formulations

With $\mathbf{y}$ being the complicating variable, we state the Benders subproblem (**SP**) below for the $\mathbf{y}$ assuming fixed values $\mathbf{\bar{y}} = (\bar{y_1}, \bar{y_2})$:

\begin{align*}
    \text{min.} &\quad 8x_1 + 12x_2 +10x_3 \\
    \text{s.t.} &\quad 2x_1 + 3x_2 + 2x_3 = 300 - 4\bar{y_1} - 5\bar{y_2} \\
    &\quad 4x_1 + 2x_2 + 3x_3 = 220 - 2\bar{y_1} - 3\bar{y_2} \\
    &\quad x_i \geq 0, \ \forall i = 1, \cdots, 3
\end{align*}

We define the dual variable $\mathbf{u} = (u_1, u_2)$ to associate with the constraints in the (**SP**).
The dual subproblem (**DSP**) could then be stated as follows:

\begin{align*}
    \text{max.} &\quad (300 - 4\bar{y_1} - 5\bar{y_2}) u_1 + (220 - 2\bar{y_1} - 3\bar{y_2}) u_2 \\
    \text{s.t.} &\quad 2u_1 + 4u_2 \leq 8\\
    &\quad 3u_1 + 2u_2 \leq 12 \\
    &\quad 2u_1 + 3u_2 \leq 10 \\
    &\quad u_1, u_2\  \text{unrestricted}
\end{align*}

The (**RBMP**) can be stated as below.
Note that $\mathbf{u} = (0, 0)$ is a feasible solution to (**DSP**) and the corresponding objective value is 0, which is the reason we restrict the variable $g$ to be nonnegative.

\begin{align*}
    \text{min.} &\quad 15 y_1 + 18 y_2 + g \\
    \text{s.t.} &\quad  y_1, y_2 \geq 0 \\
    &\quad g \geq 0
\end{align*}

### Benders decomposition step by step

Benders decomposition defines a problem solving process in which the restricted Benders master problem and the dual subproblem interact iteratively to identify the optimal solution or conclude infeasibility/unboundedness.
To facilitate our understanding of the process, we demonstrate in this section the workings of Benders decomposition by solving the target LP problem step by step.

@lst-bd-rbmp shows the codes that initialize the lower bound `lb`, upper bound `ub` and threshold value `eps`.
Furthermore, the restricted Benders master problem `rbmp` is created with three variables and a minimizing objective function.
Note that no constraints are yet included in the model at this moment.

In [110]:
#| lst-label: lst-bd-rbmp
#| lst-cap: Gurobi solver setup and restricted master problem initialization

import numpy as np
import gurobipy as gp
from gurobipy import GRB

# initialize lower/upper bounds and threshold value
lb = -GRB.INFINITY
ub = GRB.INFINITY
eps = 1.0e-5

# create restricted Benders master problem
env = gp.Env('benders', empty=True) # <1>
env.setParam('OutputFlag', 0)
env.start()
rbmp = gp.Model(env=env, name='RBMP')

# create decision variables
y1 = rbmp.addVar(vtype=GRB.CONTINUOUS, lb=0, name='y1')
y2 = rbmp.addVar(vtype=GRB.CONTINUOUS, lb=0, name='y2')
g = rbmp.addVar(vtype=GRB.CONTINUOUS, lb=0, name='g')

# create objective
rbmp.setObjective(15*y1 + 18*y2 + g, GRB.MINIMIZE)

@lst-bd-dsp initializes the Benders subproblem with two decision variables and three constraints.

In [111]:
#| lst-label: lst-bd-dsp
#| lst-cap: Dual subproblem initialization

# create dual subproblem
dsp = gp.Model(env=env, name='DSP')

# create decision variables
u1 = dsp.addVar(vtype=GRB.CONTINUOUS, name='u1')
u2 = dsp.addVar(vtype=GRB.CONTINUOUS, name='u2')

# create objective function
dsp.setObjective(300*u1 + 220*u2)

# create constraints
dsp.addConstr(2*u1 + 4*u2 <= 8, name='c1')
dsp.addConstr(3*u1 + 2*u2 <= 12, name='c2')
dsp.addConstr(2*u1 + 3*u2 <= 10, name='c3')

dsp.update()

In @lst-bd-iter1-rbmp, we solve the (**RBMP**) for the first time.
It has an optimal solution with $(\bar{y_1}, \bar{y_2}, \bar{g}) = (0, 0, 0)$ and optimal objective value of 0.
This is expected as all the variables assume their minimal possible values in order to minimize the objective function.
This objective value also serves as the new lower bound.

In [112]:
#| lst-label: lst-bd-iter1-rbmp
#| lst-cap: Iteration 1 - solving the restricted Benders master problem

rbmp.optimize()

if rbmp.status == GRB.OPTIMAL:
    print(f'Optimal solution found! Objective value = {rbmp.objVal:.2f}')
    
    y1_opt, y2_opt, g_opt = y1.X, y2.X, g.X
    lb = np.max([lb, rbmp.objVal])
    
    print(f'(y1, y2, g) = ({y1_opt:.2f}, {y2_opt:.2f}, {g_opt:.2f})')
    print(f'lb={lb}, ub={ub}')
elif rbmp.status == GRB.INFEASIBLE:
    print(f'original problem is infeasible!')

Optimal solution found! Objective value = 0.00
(y1, y2, g) = (0.00, 0.00, 0.00)
lb=0.0, ub=1e+100


Given that $(\bar{y_1}, \bar{y_2}, \bar{g}) = (0, 0, 0)$, we now feed the values of $\bar{y_1}$ and $\bar{y_2}$ into the Benders dual subproblem **(DSP)** by updating its objective function, as shown in @lst-bd-iter1-dsp:

In [113]:
#| lst-label: lst-bd-iter1-dsp
#| lst-cap: Iteration 1 - solving the dual subproblem

# update objective function
dsp.setObjective((300-4*y1_opt-5*y2_opt)*u1 + 
                 (220-2*y1_opt-3*y2_opt)*u2, 
                 GRB.MAXIMIZE)
dsp.update()
dsp.optimize()

if dsp.status == GRB.OPTIMAL:
    u1_opt, u2_opt = u1.X, u2.X
    
    print(f'Optimal objective = {dsp.objVal:.2f}')
    print(f'(u1, u2) = ({u1_opt:.2f}, {u2_opt:.2f})')
    ub = np.min([ub, 15*y1_opt + 18*y2_opt + dsp.objVal])
    print(f'lb={lb}, ub={ub}')

Optimal objective = 1200.00
(u1, u2) = (4.00, 0.00)
lb=0.0, ub=1200.0


We see that the dual subproblem has an optimal solution.
The upper bound `ub` is also updated.
Since the optimal objective value of the subproblem turns out to be 1200 and is greater than $\bar{g} = 0$, which implies that an optimality cut is needed to make sure that the variable $g$ in the restricted Benders master problem reflects this newly obtained information from the subproblem.

In @lst-bd-iter2-rbmp, the new optimality cut is added to the (**RBMP**), which is then solved to optimality.

In [114]:
#| lst-label: lst-bd-iter2-rbmp
#| lst-cap: Iteration 2 - solving the restricted Benders master problem

# add optimality cut
rbmp.addConstr((300-4*y1-5*y2)*u1_opt 
               + (220-2*y1-3*y2)*u2_opt <= g, 
               name='c1')
rbmp.update()
rbmp.optimize()

if rbmp.status == GRB.OPTIMAL:
    print(f'Optimal solution found! Objective value = {rbmp.objVal:.2f}')
    
    y1_opt, y2_opt, g_opt = y1.X, y2.X, g.X
    lb = np.max([lb, rbmp.objVal])
    
    print(f'(y1, y2, g) = ({y1_opt:.2f}, {y2_opt:.2f}, {g_opt:.2f})')
    print(f'lb={lb}, ub={ub}')
elif rbmp.status == GRB.INFEASIBLE:
    print(f'original problem is infeasible!')

Optimal solution found! Objective value = 1080.00
(y1, y2, g) = (0.00, 60.00, 0.00)
lb=1080.0, ub=1200.0


Armed with the optimal solution $(\bar{y_1}, \bar{y_2}, \bar{g}) = (0, 60, 0)$, @lst-bd-iter2-dsp updates the objective function of the (**DSP**) and obtains its optimal solution.

In [115]:
#| lst-label: lst-bd-iter2-dsp
#| lst-cap: Iteration 2 - solving the dual subproblem

# update objective function
dsp.setObjective((300-4*y1_opt-5*y2_opt)*u1 
                 + (220-2*y1_opt-3*y2_opt)*u2, 
                 GRB.MAXIMIZE)
dsp.update()
dsp.optimize()

if dsp.status == GRB.OPTIMAL:
    u1_opt, u2_opt = u1.X, u2.X
    
    print(f'Optimal objective = {dsp.objVal:.2f}')
    print(f'(u1, u2) = ({u1_opt:.2f}, {u2_opt:.2f})')
    ub = np.min([ub, 15*y1_opt + 18*y2_opt + dsp.objVal])
    print(f'lb={lb}, ub={ub:.2f}')

Optimal objective = 80.00
(u1, u2) = (0.00, 2.00)
lb=1080.0, ub=1160.00


Since the dual subproblem objective value is bigger than $\bar{g}$, an optimality cut is further needed.
In @lst-bd-iter3-rbmp, we add the new cut and solve the restricted Benders master problem again.

In [116]:
#| lst-label: lst-bd-iter3-rbmp
#| lst-cap: Iteration 3 - solving the restricted Benders master problem

# add optimality cut
rbmp.addConstr((300-4*y1-5*y2)*u1_opt 
               + (220-2*y1-3*y2)*u2_opt <= g, 
               name='c2')
rbmp.update()
rbmp.optimize()

if rbmp.status == GRB.OPTIMAL:
    print(f'Optimal solution found! Objective value = {rbmp.objVal:.2f}')
    
    y1_opt, y2_opt, g_opt = y1.X, y2.X, g.X
    lb = np.max([lb, rbmp.objVal])
    
    print(f'(y1, y2, g) = ({y1_opt:.2f}, {y2_opt:.2f}, {g_opt:.2f})')
    print(f'lb={lb:.2f}, ub={ub:.2f}')
elif rbmp.status == GRB.INFEASIBLE:
    print(f'original problem is infeasible!')

Optimal solution found! Objective value = 1091.43
(y1, y2, g) = (0.00, 54.29, 114.29)
lb=1091.43, ub=1160.00


Note that a new lower bound is obtained after solving the master problem.
Since there is still a large gap between the lower bound and upper bound, we continue solving the subproblem in @lst-bd-iter3-dsp.

In [117]:
#| lst-label: lst-bd-iter3-dsp
#| lst-cap: Iteration 3 - solving the dual subproblem

# update objective function
dsp.setObjective((300-4*y1_opt-5*y2_opt)*u1 
                 + (220-2*y1_opt-3*y2_opt)*u2, 
                 GRB.MAXIMIZE)
dsp.update()
dsp.optimize()

if dsp.status == GRB.OPTIMAL:
    u1_opt, u2_opt = u1.X, u2.X
    
    print(f'Optimal objective = {dsp.objVal:.2f}')
    print(f'(u1, u2) = ({u1_opt:.2f}, {u2_opt:.2f})')
    ub = np.min([ub, 15*y1_opt + 18*y2_opt + dsp.objVal])
    print(f'lb={lb:.2f}, ub={ub:.2f}')

Optimal objective = 114.29
(u1, u2) = (0.00, 2.00)
lb=1091.43, ub=1091.43


Now that the difference between `lb` and `ub` is less than the preset threshold `eps`, we conclude that an optimal solution is reached and the computation resources are freed up.

In [134]:
# release resources
rbmp.dispose()
dsp.dispose()
env.dispose()

### Putting it together

It usually takes many iterations of solving the (**RBMP**) and (**DSP**) sequentially before reaching optimality.
Certainly we don't want to manually control the interaction between the master problem and subproblem to find the optimal solution.
Therefore, in this section, we will put everything together to come up with a control flow to help us identify the optimal solution automatically.

In [118]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from enum import Enum

class OptStatus(Enum):
    OPTIMAL = 0
    UNBOUNDED = 1
    INFEASIBLE = 2
    ERROR = 3

In [119]:
class MasterSolver:
    
    def __init__(self, env):
        self._model = gp.Model(env=env, name='RBMP')
        
        # create decision variables
        self._y1 = self._model.addVar(vtype=GRB.CONTINUOUS, lb=0, name='y1')
        self._y2 = self._model.addVar(vtype=GRB.CONTINUOUS, lb=0, name='y2')
        self._g = self._model.addVar(vtype=GRB.CONTINUOUS, lb=0, name='g')

        # create objective
        self._model.setObjective(15*self._y1 + 18*self._y2 + self._g, GRB.MINIMIZE)
        
        self._opt_obj = None
        self._opt_y1 = None
        self._opt_y2 = None
        self._opt_g = None
        
    def solve(self) -> OptStatus:
        print('-' * 50)
        print(f'Start solving master problem.')
        self._model.optimize()
        
        opt_status = None
        if self._model.status == GRB.OPTIMAL:
            opt_status = OptStatus.OPTIMAL
            self._opt_obj = self._model.objVal
            self._opt_y1 = self._y1.X
            self._opt_y2 = self._y2.X
            self._opt_g = self._g.X
            print(f'\tmaster problem is optimal.')
            print(f'\topt_obj={self._opt_obj:.2f}')
            print(f'\topt_y1={self._opt_y1:.2f}, opt_y2={self._opt_y2:.2f}, opt_g={self._opt_g:.2f}')
        elif self._model.status == GRB.INFEASIBLE:
            print(f'\tmaster problem is infeasible.')
            opt_status = OptStatus.INFEASIBLE
        else:
            print(f'\tmaster problem encountered error.')
            opt_status = OptStatus.ERROR
        
        print(f'Finish solving master problem.') 
        print('-' * 50)
        return opt_status
    
    def add_feasibility_cut(self, opt_u1, opt_u2) -> None:
        self._model.addConstr((300 - 4*self._y1 - 5*self._y2) * opt_u1 + 
                              (220 - 2*self._y1 - 3*self._y2) * opt_u2 <= 0)
        print(f'Benders feasibility cut added!')
    
    def add_optimality_cut(self, opt_u1, opt_u2) -> None:
        self._model.addConstr((300 - 4*self._y1 - 5*self._y2) * opt_u1 + 
                              (220 - 2*self._y1 - 3*self._y2) * opt_u2 <= self._g)
        print(f'Benders optimality cut added!')
    
    def clean_up(self):
        self._model.dispose()
        
    @property
    def opt_obj(self):
        return self._opt_obj
    
    @property
    def opt_y1(self):
        return self._opt_y1
    
    @property
    def opt_y2(self):
        return self._opt_y2
    
    @property
    def opt_g(self):
        return self._g

In [120]:
class DualSubprobSolver:
    
    def __init__(self, env):
        self._model = gp.Model(env=env, name='DSP')
        
        # create decision variables
        self._u1 = self._model.addVar(vtype=GRB.CONTINUOUS, name='u1')
        self._u2 = self._model.addVar(vtype=GRB.CONTINUOUS, name='u2')
        
        # create constraints
        self._model.addConstr(2*self._u1 + 4*self._u2 <= 8, name='c1')
        self._model.addConstr(3*self._u1 + 2*self._u2 <= 12, name='c2')
        self._model.addConstr(2*self._u1 + 3*self._u2 <= 10, name='c3')
        
        self._model.setObjective(1, GRB.MAXIMIZE)
        self._model.update()
        
        self._opt_obj = None
        self._opt_u1 = None
        self._opt_u2 = None
    
    def solve(self):
        print('-' * 50)
        print(f'Start solving dual subproblem.')
        self._model.optimize()
        
        status = None
        if self._model.status == GRB.OPTIMAL:
            self._opt_obj = self._model.objVal
            self._opt_u1 = self._u1.X
            self._opt_u2 = self._u2.X
            status = OptStatus.OPTIMAL
            print(f'\tdual subproblem is optimal.')
            print(f'\topt_obj={self._opt_obj:.2f}')
            print(f'\topt_y1={self._opt_u1:.2f}, opt_y2={self._opt_u2:.2f}')
        elif self._model.status == GRB.UNBOUNDED:
            status = OptStatus.UNBOUNDED
        else:
            status = OptStatus.ERROR
        
        print(f'Finish solving dual subproblem.')
        print('-' * 50)
        return status
    
    def update_objective(self, opt_y1, opt_y2):
        self._model.setObjective((300-4*opt_y1-5*opt_y2)*self._u1 + (220-2*opt_y1-3*opt_y2)*self._u2, GRB.MAXIMIZE)
        print(f'dual subproblem objective updated!')
    
    def clean_up(self):
        self._model.dispose()
        
    @property
    def opt_obj(self):
        return self._opt_obj
    
    @property
    def opt_u1(self):
        return self._opt_u1
    
    @property
    def opt_u2(self):
        return self._opt_u2

In [121]:
class BendersDecomposition:
    
    def __init__(self, master_solver, dual_subprob_solver):
        self._master_solver = master_solver
        self._dual_subprob_solver = dual_subprob_solver
        
    
    def optimize(self) -> OptStatus:
        eps = 1.0e-5
        lb = -np.inf
        ub = np.inf
        
        while True:
            # solve master problem
            master_status = self._master_solver.solve()
            if master_status == OptStatus.INFEASIBLE:
                return OptStatus.INFEASIBLE
            
            # update lower bound
            lb = np.max([lb, self._master_solver.opt_obj])
            print(f'Bounds: lb={lb:.2f}, ub={ub:.2f}')
            
            opt_y1 = self._master_solver.opt_y1
            opt_y2 = self._master_solver.opt_y2
            
            # solve subproblem
            self._dual_subprob_solver.update_objective(opt_y1, opt_y2)
            dsp_status = self._dual_subprob_solver.solve()
            
            if dsp_status == OptStatus.OPTIMAL:
                # update upper bound
                opt_obj = self._dual_subprob_solver.opt_obj
                ub = np.min([ub, 15*opt_y1 + 18*opt_y2 + opt_obj])
                print(f'Bounds: lb={lb:.2f}, ub={ub:.2f}')
                
                if ub - lb <= eps:
                    break
                
                opt_u1 = self._dual_subprob_solver.opt_u1
                opt_u2 = self._dual_subprob_solver.opt_u2
                self._master_solver.add_optimality_cut(opt_u1, opt_u2) 
            elif dsp_status == OptStatus.UNBOUNDED:
                opt_u1 = self._dual_subprob_solver.opt_u1
                opt_u2 = self._dual_subprob_solver.opt_u2
                self._master_solver.add_feasibility_cut(opt_u1, opt_u2) 
            

In [122]:
env = gp.Env('benders')
env.setParam("OutputFlag",0)
master_solver = MasterSolver(env)
dual_subprob_solver = DualSubprobSolver(env)

benders_decomposition = BendersDecomposition(master_solver, dual_subprob_solver)
benders_decomposition.optimize()

Set parameter Username
Set parameter LogFile to value "benders"
--------------------------------------------------
Start solving master problem.
	master problem is optimal.
	opt_obj=0.00
	opt_y1=0.00, opt_y2=0.00, opt_g=0.00
Finish solving master problem.
--------------------------------------------------
Bounds: lb=0.00, ub=inf
dual subproblem objective updated!
--------------------------------------------------
Start solving dual subproblem.
	dual subproblem is optimal.
	opt_obj=1200.00
	opt_y1=4.00, opt_y2=0.00
Finish solving dual subproblem.
--------------------------------------------------
Bounds: lb=0.00, ub=1200.00
Benders optimality cut added!
--------------------------------------------------
Start solving master problem.
	master problem is optimal.
	opt_obj=1080.00
	opt_y1=0.00, opt_y2=60.00, opt_g=0.00
Finish solving master problem.
--------------------------------------------------
Bounds: lb=1080.00, ub=1200.00
dual subproblem objective updated!
--------------------------

### A generic solver

In this section, we will create a more generic Benders decomposition based solver for linear programming problems.

In [135]:
class GenericLpMasterSolver:
    
    def __init__(self, f: np.array, B: np.array, b: np.array):
        # save data
        self._f = f
        self._B = B
        self._b = b 
        
        # env and model
        self._env = gp.Env('MasterEnv')
        self._env.setParam("OutputFlag",0)
        self._model = gp.Model(env=self._env, name='MasterSolver')
        
        # create variables
        self._num_y_vars = len(f)
        self._y = self._model.addVars(self._num_y_vars, lb=0, vtype=GRB.CONTINUOUS, name='y')
        self._g = self._model.addVar(vtype=GRB.CONTINUOUS, lb=0, name='g')
        
        # create objective
        self._model.setObjective(gp.quicksum(self._f[i] * self._y.get(i) 
                                             for i in range(self._num_y_vars)) + self._g, 
                                 GRB.MINIMIZE)
        self._model.update()
        
        self._opt_obj = None
        self._opt_obj_y = None
        self._opt_y = None
        self._opt_g = None
        
    def solve(self) -> OptStatus:
        print('-' * 50)
        print(f'Start solving master problem.')
        print(self._model.display())
        self._model.optimize()
        
        opt_status = None
        if self._model.status == GRB.OPTIMAL:
            opt_status = OptStatus.OPTIMAL
            self._opt_obj = self._model.objVal
            self._opt_y = {
                i: self._y.get(i).X
                for i in range(self._num_y_vars)
            }
            self._opt_g = self._g.X
            self._opt_obj_y = self._opt_obj - self._opt_g
            print(f'\tmaster problem is optimal.')
            print(f'\topt_obj={self._opt_obj:.2f}')
            print(f'\topt_g={self._opt_g:.2f}')
            # for i in range(self._num_y_vars):
            #     print(f'\topt_y{i}={self._opt_y.get(i)}')
        elif self._model.status == GRB.INFEASIBLE:
            print(f'\tmaster problem is infeasible.')
            opt_status = OptStatus.INFEASIBLE
        else:
            print(f'\tmaster problem encountered error.')
            opt_status = OptStatus.ERROR
        
        print(f'Finish solving master problem.') 
        print('-' * 50)
        return opt_status
    
    def add_feasibility_cut(self, opt_u: dict) -> None:
        constr_expr = [
            opt_u.get(u_idx) * (self._b[u_idx] - gp.quicksum(self._B[u_idx][j] * self._y.get(j) 
                                                   for j in range(self._num_y_vars)))
            for u_idx in opt_u.keys()
        ]
        self._model.addConstr(gp.quicksum(constr_expr) <= 0)
        print(f'Benders feasibility cut added!')
    
    def add_optimality_cut(self, opt_u: dict) -> None:
        constr_expr = [
            opt_u.get(u_idx) * (self._b[u_idx] - gp.quicksum(self._B[u_idx][j] * self._y.get(j) 
                                                   for j in range(self._num_y_vars)))
            for u_idx in opt_u.keys()
        ]
        self._model.addConstr(gp.quicksum(constr_expr) <= self._g)
        self._model.update()
        print(self._model.display())
        print(f'Benders optimality cut added!')
    
    def clean_up(self):
        self._model.dispose()
        self._env.dispose()
        
    @property
    def f(self):
        return self._f
        
    @property
    def opt_obj(self):
        return self._opt_obj
    
    @property
    def opt_obj_y(self):
        return self._opt_obj_y
    
    @property
    def opt_y(self):
        return self._opt_y
    
    @property
    def opt_g(self):
        return self._g

In [136]:
class GenericLpSubprobSolver:
    
    def __init__(self, A: np.array, c: np.array, B: np.array, b: np.array):
        # save data
        self._A = A 
        self._c = c 
        self._b = b 
        self._B = B
        
        # env and model
        self._env = gp.Env('SubprobEnv')
        self._env.setParam("OutputFlag",0)
        self._model = gp.Model(env=self._env, name='SubprobSolver')

        # create variables
        self._num_vars = len(b)
        self._u = self._model.addVars(self._num_vars, vtype=GRB.CONTINUOUS, name='u')

        # create constraints
        for c_idx in range(len(c)):
            self._model.addConstr(gp.quicksum(A[:,c_idx][i] * self._u.get(i) 
                                              for i in range(len(b))) <= c[c_idx])
        
        self._opt_obj = None
        self._opt_u = None
        self._extreme_ray = None
        
    def solve(self):
        print('-' * 50)
        print(f'Start solving dual subproblem.')
        self._model.setParam(GRB.Param.DualReductions, 0)
        self._model.setParam(GRB.Param.InfUnbdInfo, 1)
        self._model.optimize()
        
        status = None
        if self._model.status == GRB.OPTIMAL:
            self._opt_obj = self._model.objVal
            self._opt_u = {
                i: self._u.get(i).X
                for i in range(self._num_vars)
            }
            status = OptStatus.OPTIMAL
            print(f'\tdual subproblem is optimal.')
            print(f'\topt_obj={self._opt_obj:.2f}')
            # for i in range(self._num_vars):
            #     print(f'\topt_u{i}={self._opt_u.get(i)}')
        elif self._model.status == GRB.UNBOUNDED:
            status = OptStatus.UNBOUNDED
            self._extreme_ray = {
                i: self._u.get(i).UnbdRay
                for i in range(self._num_vars)
            }
            print(f'dual subproblem is unbounded')
            # for i in range(self._num_vars):
            #     print(f'\topt_u{i}={self._extreme_ray.get(i)}')
        else:
            status = OptStatus.ERROR
        
        print(f'Finish solving dual subproblem.')
        print('-' * 50)
        return status

    def update_objective(self, opt_y: dict):
        obj_expr = [
            self._u.get(u_idx) * (self._b[u_idx] - sum(self._B[u_idx][j] * opt_y.get(j) 
                                                    for j in range(len(opt_y))))
            for u_idx in range(self._num_vars)
        ]
        self._model.setObjective(gp.quicksum(obj_expr), GRB.MAXIMIZE)
        print(f'dual subproblem objective updated!')
    
    def clean_up(self):
        self._model.dispose()
        self._env.dispose()
        
    @property
    def opt_obj(self):
        return self._opt_obj
    
    @property
    def opt_u(self):
        return self._opt_u
    
    @property
    def extreme_ray(self):
        return self._extreme_ray

In [137]:
class GenericBendersSolver:
    
    def __init__(self, master_solver, dual_subprob_solver):
        self._master_solver = master_solver
        self._dual_subprob_solver = dual_subprob_solver
        
    
    def optimize(self,) -> OptStatus:
        eps = 1.0e-5
        lb = -np.inf
        ub = np.inf
        
        while True:
            # solve master problem
            master_status = self._master_solver.solve()
            if master_status == OptStatus.INFEASIBLE:
                return OptStatus.INFEASIBLE
            
            # update lower bound
            lb = np.max([lb, self._master_solver.opt_obj])
            print(f'Bounds: lb={lb:.2f}, ub={ub:.2f}')
            
            opt_y = self._master_solver.opt_y
            
            # solve subproblem
            self._dual_subprob_solver.update_objective(opt_y)
            dsp_status = self._dual_subprob_solver.solve()
            
            if dsp_status == OptStatus.OPTIMAL:
                # update upper bound
                opt_obj = self._dual_subprob_solver.opt_obj
                opt_obj_y = self._master_solver.opt_obj_y
                ub = np.min([ub, opt_obj_y + opt_obj])
                print(f'Bounds: lb={lb:.2f}, ub={ub:.2f}')
                
                if ub - lb <= eps:
                    break
                
                opt_u = self._dual_subprob_solver.opt_u
                self._master_solver.add_optimality_cut(opt_u) 
            elif dsp_status == OptStatus.UNBOUNDED:
                extreme_ray = self._dual_subprob_solver.extreme_ray
                self._master_solver.add_feasibility_cut(extreme_ray) 

In [138]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

c = np.array([8, 12, 10])
f = np.array([15, 18])
A = np.array([
    [2, 3, 2],
    [4, 2, 3]
])
B = np.array([
    [4, 5],
    [2, 3],
])
b = np.array([300, 220])

master_solver = GenericLpMasterSolver(f, B, b)
dual_subprob_solver = GenericLpSubprobSolver(A, c, B, b)

benders_solver = GenericBendersSolver(master_solver, dual_subprob_solver)
benders_solver.optimize()

Set parameter Username
Set parameter LogFile to value "MasterEnv"
Set parameter Username
Set parameter LogFile to value "SubprobEnv"
--------------------------------------------------
Start solving master problem.
None
	master problem is optimal.
	opt_obj=0.00
	opt_g=0.00
Finish solving master problem.
--------------------------------------------------
Bounds: lb=0.00, ub=inf
dual subproblem objective updated!
--------------------------------------------------
Start solving dual subproblem.
	dual subproblem is optimal.
	opt_obj=1200.00
Finish solving dual subproblem.
--------------------------------------------------
Bounds: lb=0.00, ub=1200.00
None
Benders optimality cut added!
--------------------------------------------------
Start solving master problem.
None
	master problem is optimal.
	opt_obj=1080.00
	opt_g=0.00
Finish solving master problem.
--------------------------------------------------
Bounds: lb=1080.00, ub=1200.00
dual subproblem objective updated!
---------------------

In [139]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

c = np.array([1, 1, 1, 1, 1, 1])
f = np.array([1, 1, 1, 1])
A = np.array([
    [1, 1, 1, 1, 1, 1]
])
B = np.array([
    [1, 1, 1, 1]
])
b = np.array([1])

master_solver = GenericLpMasterSolver(f, B, b)
dual_subprob_solver = GenericLpSubprobSolver(A, c, B, b)

benders_solver = GenericBendersSolver(master_solver, dual_subprob_solver)
benders_solver.optimize()

Set parameter Username
Set parameter LogFile to value "MasterEnv"
Set parameter Username
Set parameter LogFile to value "SubprobEnv"
--------------------------------------------------
Start solving master problem.
None
	master problem is optimal.
	opt_obj=0.00
	opt_g=0.00
Finish solving master problem.
--------------------------------------------------
Bounds: lb=0.00, ub=inf
dual subproblem objective updated!
--------------------------------------------------
Start solving dual subproblem.
	dual subproblem is optimal.
	opt_obj=1.00
Finish solving dual subproblem.
--------------------------------------------------
Bounds: lb=0.00, ub=1.00
None
Benders optimality cut added!
--------------------------------------------------
Start solving master problem.
None
	master problem is optimal.
	opt_obj=1.00
	opt_g=0.00
Finish solving master problem.
--------------------------------------------------
Bounds: lb=1.00, ub=1.00
dual subproblem objective updated!
------------------------------------

In [140]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp

class GurobiLpSolver:
    
    def __init__(self, c, f, A, B, b):
        self._env = gp.Env('GurobiEnv')
        self._model = gp.Model(env=self._env, name='GurobiLpSolver')
        
        # prepare data
        self._obj_coeff = np.concatenate((c, f))
        print(self._obj_coeff)
        self._constr_mat = np.concatenate((A, B), axis=1)
        print(self._constr_mat)
        self._rhs = b
        self._num_vars = len(self._obj_coeff)
        self._num_constrs = len(b)
        
        # create decision variables
        self._vars = self._model.addMVar(self._num_vars, vtype=GRB.CONTINUOUS, lb=0)
        
        # create constraints
        self._constrs = self._model.addConstr(self._constr_mat@self._vars == self._rhs)
        
        # create objective
        self._model.setObjective(self._obj_coeff @ self._vars, GRB.MINIMIZE)
    
    def optimize(self):
        self._model.update()
        print(self._model.display())
        self._model.optimize()
        pass
    
    def clean_up(self):
        
        pass

In [141]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

np.random.seed(142)
c = np.random.randint(2, 6, size=20)
f = np.random.randint(1, 15, size=10)
A = np.random.randint(2, 6, size=(20, 20))
B = np.random.randint(2, 26, size=(20, 10))
b = np.random.randint(20, 50, size=20)

model = GurobiLpSolver(c, f, A, B, b)
model.optimize()

Set parameter Username
Set parameter LogFile to value "GurobiEnv"
[ 3  3  5  5  2  4  3  2  3  3  4  2  5  5  2  5  4  4  5  5  3  6  8 12
  3  5  4  4  9  3]
[[ 2  4  3  5  5  2  5  4  5  3  3  4  2  2  2  2  3  2  3  3 11 11 25 20
   9  5  7  8  9  2]
 [ 5  2  2  4  3  5  4  2  3  5  4  3  2  5  2  3  5  4  5  4 23 19 23 19
  12 12 13 15 19 19]
 [ 2  3  2  2  5  5  4  5  5  2  5  2  5  4  5  4  3  5  3  3 11 10  8 20
  17  9 25 25 10 12]
 [ 3  5  4  3  4  2  4  2  4  2  4  2  3  3  2  5  3  2  4  2 16 23 18 20
  20 25 14 10  2  6]
 [ 5  4  3  4  3  4  5  4  4  2  5  4  2  3  2  2  5  2  4  3  3  8 10  4
  22 25 11 15 15  9]
 [ 5  5  5  3  2  5  2  2  3  4  2  5  4  4  2  5  4  5  5  4 21 14 22 19
  15 19 16  8 22 23]
 [ 4  2  2  5  2  5  5  4  2  3  2  3  2  5  4  3  3  4  5  3 18  8 11  2
  19 23 23  8 18 25]
 [ 3  2  4  5  3  2  3  5  3  4  5  2  5  4  2  4  2  4  3  5 20 19 13 24
  19  7  4 15 24  3]
 [ 4  4  2  3  2  2  5  5  2  3  5  5  4  5  3  4  2  2  4  2 19  6 20 16
   5 14

In [142]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

np.random.seed(142)
c = np.random.randint(2, 6, size=20)
f = np.random.randint(1, 15, size=10)
A = np.random.randint(2, 6, size=(20, 20))
B = np.random.randint(2, 26, size=(20, 10))
b = np.random.randint(20, 50, size=20)

master_solver = GenericLpMasterSolver(f, B, b)
dual_subprob_solver = GenericLpSubprobSolver(A, c, B, b)

benders_solver = GenericBendersSolver(master_solver, dual_subprob_solver)
benders_solver.optimize()

Set parameter Username
Set parameter LogFile to value "MasterEnv"
Set parameter Username
Set parameter LogFile to value "SubprobEnv"
--------------------------------------------------
Start solving master problem.
None
	master problem is optimal.
	opt_obj=0.00
	opt_g=0.00
Finish solving master problem.
--------------------------------------------------
Bounds: lb=0.00, ub=inf
dual subproblem objective updated!
--------------------------------------------------
Start solving dual subproblem.
	dual subproblem is optimal.
	opt_obj=28.16
Finish solving dual subproblem.
--------------------------------------------------
Bounds: lb=0.00, ub=28.16
None
Benders optimality cut added!
--------------------------------------------------
Start solving master problem.
None
	master problem is optimal.
	opt_obj=6.97
	opt_g=0.00
Finish solving master problem.
--------------------------------------------------
Bounds: lb=6.97, ub=28.16
dual subproblem objective updated!
---------------------------------

In [131]:
c

array([3, 3, 5, 5, 2, 4, 3, 2, 3, 3, 4, 2, 5, 5, 2, 5, 4, 4, 5, 5])

In [132]:
np.random.randint(1, 5, size=(2, 10))

array([[3, 2, 2, 4, 2, 3, 1, 2, 3, 2],
       [3, 4, 3, 3, 2, 2, 1, 4, 3, 4]])

### Implementation with callbacks

### Implementation with SCIP

In [133]:
from pyscipopt import Model
from pyscipopt import quicksum
from pyscipopt import SCIP_PARAMSETTING

# Create a model
model = Model("simple_lp")

# Define variables
x1 = model.addVar(lb=0, vtype="C", name="x1")
x2 = model.addVar(lb=0, vtype="C", name="x2")

# Set objective function
model.setObjective(x1 + x2, "maximize")

# Add constraints
# model.addCons(2 * x1 + x2 >= 1, "constraint1")
model.addCons(x1 + x2 >= 2, "constraint2")

# Solve the model
model.setPresolve(SCIP_PARAMSETTING.OFF)
model.setHeuristics(SCIP_PARAMSETTING.OFF)
model.disablePropagation()
model.optimize()

# Print results
status = model.getStatus()
print(f'status = {status}')
if model.getStatus() == "optimal":
    print("Optimal solution found.")
    print(f"x1: {model.getVal(x1):.2f}")
    print(f"x2: {model.getVal(x2):.2f}")
    print(f"Objective value: {model.getObjVal():.2f}")
    hasRay = model.hasPrimaryRay()
    print(hasRay)
elif model.getStatus() == 'unbounded':
    hasRay = model.hasPrimalRay()
    print(f'hasRay={hasRay}')
    ray = model.getPrimalRay()
    print(f'ray={ray}')
    
else:
    print("Model could not be solved.")


status = unbounded
hasRay=True
ray=[0.5, 0.5]
presolving:
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) symmetry computation finished: 1 generators found (max: 1500, log10 of symmetry group size: 0.3) (symcode time: 0.00)
   (0.0s) no symmetry on binary variables present.
presolving (0 rounds: 0 fast, 0 medium, 0 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 2 variables (0 bin, 0 int, 0 impl, 2 cont) and 2 constraints
      2 constraints of type <linear>
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
* 0.0s|     1 |     0 |     2 |     - |    LP  |   0 |   2 |   2 |   2 |   1 |  0 |   0 |   0 |      --      |      --      |   0.00%| unknown

SCIP Status

### Testing